In [23]:
import tkinter as tk
from tkinter import ttk
import mysql.connector as connector

def connect_to_database():
    global connection, cursor
    try:
        # Retrieve the password from the entry widget
        password = password_entry.get()

        # Establish connection between Python and MySQL database via connector API
        connection = connector.connect(
            host="localhost",
            user="DisenchantmentAdmin",  # use your own
            password=password,  # use the entered password
            database="testdisenchantment",  # set the default database
            raise_on_warnings=True
        )

        print("Connection between MySQL and Python is established.\n")
        connection_label.config(text="Connected")
        open_second_window()
    except connector.Error as err:
        print(f"Error connecting to the database: {err}")

def close_connection():
    global connection
    try:
        # Close the database connection
        connection.close()
        print("Connection to the database is closed.\n")
        connection_label.config(text="Not Connected")
    except connector.Error as err:
        print(f"Error closing the connection: {err}")

#----------------------------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------------------------------

def open_second_window():
    second_window = tk.Toplevel()
    second_window.title("Second Window")

    # Create the "Elements" button in the second window
    elements_button = tk.Button(second_window, text="Elements", command=open_admin_elements_window)
    elements_button.pack()

    # Create the "Scenes" button in the second window
    scenes_button = tk.Button(second_window, text="Scenes", command=open_admin_scenes_window)
    scenes_button.pack()

    # Create the "Episodes" button in the second window
    scenes_button = tk.Button(second_window, text="Episodes", command=open_admin_episodes_window)
    scenes_button.pack()

#-------------------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------

def add_episode(tree, part_id_entry, episode_name_entry, synopsis_entry):
    # Get the values from text boxes
    episode_name = episode_name_entry.get()
    part_id = part_id_entry.get()
    synopsis = synopsis_entry.get()

    try:
        # Establish connection between Python and MySQL database via connector API
        connection = connector.connect(
            host="localhost",
            user="DisenchantmentAdmin",  # use your own
            password=password_entry.get(),  # use the entered password
            database="testdisenchantment",  # set the default schema
            raise_on_warnings=True
        )

        # Create a cursor object to execute SQL statements
        cursor = connection.cursor()

        # Insert the episode into the 'episode' table
        insert_query = "INSERT INTO episode (PartID, EpisodeName, Synopsis) VALUES (%s, %s, %s)"
        cursor.execute(insert_query, (part_id, episode_name, synopsis))
        connection.commit()
        print("Episode added successfully.")

        # Fetch data from the updated 'episode' table
        cursor.execute("SELECT EpisodeID, EpisodeName, PartID, Synopsis FROM episode")

        # Clear the tree view
        tree.delete(*tree.get_children())

        # Insert the data into the tree view
        for row in cursor:
            tree.insert("", tk.END, values=row)

    except connector.Error as err:
        print(f"Failed to add episode: {err}")

    finally:
        # Close the cursor and the connection
        cursor.close()
        connection.close()
##-----------------------------------------------------------------------------------------------------------------------
##------------------------------------------------------------------------------------------------------------------------

def open_admin_episodes_window():
    admin_episodes_window = tk.Toplevel(window)
    admin_episodes_window.title("Admin Episodes")

    # Create the tree view
    tree = ttk.Treeview(admin_episodes_window, columns=("EpisodeID", "EpisodeName", "PartID", "Synopsis"))
    tree.heading("EpisodeID", text="Episode ID")
    tree.heading("PartID", text="Part ID")
    tree.heading("EpisodeName", text="Episode Name")
    tree.heading("Synopsis", text="Synopsis")
    tree.pack()

    # Fetch data from the 'episodes' table
    try:
        # Establish connection between Python and MySQL database via connector API
        connection = connector.connect(
            host="localhost",
            user="DisenchantmentAdmin",  # use your own
            password=password_entry.get(),  # use the entered password
            database="testdisenchantment",  # set the default schema
            raise_on_warnings=True
        )

        # Create a cursor object to execute SQL statements
        cursor = connection.cursor()

        # Fetch data from the 'episodes' table
        cursor.execute("SELECT EpisodeID, EpisodeName, PartID, EpisodeName, Synopsis FROM episode")

        # Insert the data into the tree view
        for row in cursor:
            tree.insert("", tk.END, values=row)

    except connector.Error as err:
        print(f"Error fetching data from the 'episodes' table: {err}")

    finally:
        # Close the cursor and the connection
        cursor.close()
        connection.close()

    # Create frames for interactive text boxes and labels
    text_boxes_frame = tk.Frame(admin_episodes_window)
    text_boxes_frame.pack(pady=20)

    # Create the labels for text boxes
    part_id_label = tk.Label(text_boxes_frame, text="Part ID:")
    part_id_label.grid(row=0, column=0)
    episode_name_label = tk.Label(text_boxes_frame, text="Episode Name:")
    episode_name_label.grid(row=1, column=0)
    synopsis_label = tk.Label(text_boxes_frame, text="Synopsis:")
    synopsis_label.grid(row=2, column=0)

    # Create the text boxes
    part_id_entry = tk.Entry(text_boxes_frame)
    part_id_entry.grid(row=0, column=1)
    episode_name_entry = tk.Entry(text_boxes_frame)
    episode_name_entry.grid(row=1, column=1)
    synopsis_entry = tk.Entry(text_boxes_frame)
    synopsis_entry.grid(row=2, column=1)
    

    # Create the "Add" button
    add_button = tk.Button(admin_episodes_window, text="Add", command=lambda: add_episode(tree, part_id_entry, episode_name_entry, synopsis_entry))
    add_button.pack()

    # Create the label for displaying status
    status_label = tk.Label(admin_episodes_window, text="")
    status_label.pack()

    #---------------------------------------------------------------------------------------------------------------------------------------------------------------
    #--------------------------------------------------------------------------------------------------------------------------------------------------------
def open_admin_elements_window():
    admin_elements_window = tk.Toplevel(window)
    admin_elements_window.title("Admin Elements")

    # Create the tree view
    tree = ttk.Treeview(admin_elements_window, columns=("ElementID", "ElementReferenceName", "ElementDescription", "ElementType"))
    tree.heading("ElementID", text="Element ID")
    tree.heading("ElementReferenceName", text="Reference Name")
    tree.heading("ElementDescription", text="Description")
    tree.heading("ElementType", text="Element Type")
    tree.pack()

    # Create frames for interactive text boxes and labels
    text_boxes_frame = tk.Frame(admin_elements_window)
    text_boxes_frame.pack(pady=20)

    # Create the labels for text boxes
    element_ref_label = tk.Label(text_boxes_frame, text="ElementReferenceName:")
    element_ref_label.grid(row=0, column=0)
    element_desc_label = tk.Label(text_boxes_frame, text="ElementDescription:")
    element_desc_label.grid(row=1, column=0)
    element_type_label = tk.Label(text_boxes_frame, text="ElementType:")
    element_type_label.grid(row=2, column=0)

    # Create the text boxes
    element_ref_entry = tk.Entry(text_boxes_frame)
    element_ref_entry.grid(row=0, column=1)
    element_desc_entry = tk.Entry(text_boxes_frame)
    element_desc_entry.grid(row=1, column=1)
    element_type_entry = tk.Entry(text_boxes_frame)
    element_type_entry.grid(row=2, column=1)

    # Create the "Add" button
    add_button = tk.Button(admin_elements_window, text="Add", command=lambda: add_element(tree, element_ref_entry, element_desc_entry, element_type_entry))
    add_button.pack()

    # Create the label for displaying status
    status_label = tk.Label(admin_elements_window, text="")
    status_label.pack()

    def add_element(tree, element_ref_entry, element_desc_entry, element_type_entry):
        # Get the values from text boxes
        element_reference_name = element_ref_entry.get()
        element_description = element_desc_entry.get()
        element_type_id = element_type_entry.get()

        try:
            # Establish connection between Python and MySQL database via connector API
            connection = connector.connect(
                host="localhost",
                user="DisenchantmentAdmin",  # use your own
                password=password_entry.get(),  # use the entered password
                database="testdisenchantment",  # set the default schema
                raise_on_warnings=True
            )

            # Create a cursor object to execute SQL statements
            cursor = connection.cursor()

            # Check if the element_reference_name already exists
            cursor.execute("SELECT COUNT(*) FROM element WHERE ElementReferenceName=%s", (element_reference_name,))
            result = cursor.fetchone()

            if result[0] == 0:
                # Call the AddElement procedure with the provided values
                cursor.callproc("AddElement", (element_type_id, element_reference_name, element_description))
                connection.commit()
                print("Element added successfully.")
                status_label.config(text="Element added successfully.", fg="green")
                element_ref_entry.delete(0, tk.END)
                element_desc_entry.delete(0, tk.END)
                element_type_entry.delete(0, tk.END)
            else:
                print("Element already exists.")
                status_label.config(text="Element already exists.", fg="red")

            # Fetch data from the 'element' table
            cursor.execute("SELECT ElementID, ElementReferenceName, ElementDescription, ElementType FROM element")

            # Insert the data into the tree view
            tree.delete(*tree.get_children())

            for row in cursor:
                tree.insert("", tk.END, values=row)

        except connector.Error as err:
            print(f"Failed to add element: {err}")
            status_label.config(text="Failed to add element.", fg="red")

        finally:
            # Close the cursor and the connection
            cursor.close()
            connection.close()

    # Establish connection between Python and MySQL database via connector API
    connection = connector.connect(
        host="localhost",
        user="DisenchantmentAdmin",  # use your own
        password=password_entry.get(),  # use the entered password
        database="testdisenchantment",  # set the default schema
        raise_on_warnings=True
    )

    try:
        # Create a cursor object to execute SQL statements
        cursor = connection.cursor()

        # Fetch data from the 'element' table
        cursor.execute("SELECT ElementID, ElementReferenceName, ElementDescription, ElementType FROM element")

        # Insert the data into the tree view
        tree.delete(*tree.get_children())

        for row in cursor:
            tree.insert("", tk.END, values=row)

    except connector.Error as err:
        print(f"Error fetching data from the 'element' table: {err}")
#----------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------------
def open_admin_scenes_window():
    admin_scenes_window = tk.Toplevel(window)
    admin_scenes_window.title("Admin Scenes")

    # Create the tree view
    tree = ttk.Treeview(admin_scenes_window, columns=("SceneID", "SceneName", "SceneDescription", "EpisodeID", "Timestamp"))
    tree.heading("SceneID", text="Scene ID")
    tree.heading("SceneName", text="Scene Name")
    tree.heading("SceneDescription", text="Description")
    tree.heading("EpisodeID", text="Episode ID")
    tree.heading("Timestamp", text="Timestamp")
    tree.pack()

    # Create frames for interactive text boxes and labels
    text_boxes_frame = tk.Frame(admin_scenes_window)
    text_boxes_frame.pack(pady=20)

    # Create the labels for text boxes
    scene_name_label = tk.Label(text_boxes_frame, text="Scene Name:")
    scene_name_label.grid(row=0, column=0)
    scene_desc_label = tk.Label(text_boxes_frame, text="Scene Description:")
    scene_desc_label.grid(row=1, column=0)
    episode_ID_label = tk.Label(text_boxes_frame, text="EpisodeID:")
    episode_ID_label.grid(row=2, column=0)
    timestamp_label = tk.Label(text_boxes_frame, text="Timestamp:")
    timestamp_label.grid(row=3, column=0)

    # Create the text boxes
    scene_name_entry = tk.Entry(text_boxes_frame)
    scene_name_entry.grid(row=0, column=1)
    scene_desc_entry = tk.Entry(text_boxes_frame)
    scene_desc_entry.grid(row=1, column=1)
    episode_ID_entry = tk.Entry(text_boxes_frame)
    episode_ID_entry.grid(row=2, column=1)
    timestamp_entry = tk.Entry(text_boxes_frame)
    timestamp_entry.grid(row=3, column=1)

    # Create the "Add" button
    add_button = tk.Button(admin_scenes_window, text="Add", command=lambda: add_scene(tree, scene_name_entry, scene_desc_entry, episode_ID_entry, timestamp_entry))
    add_button.pack()

    # Create the label for displaying status
    status_label = tk.Label(admin_scenes_window, text="")
    status_label.pack()

    def add_scene(tree, scene_name_entry, scene_desc_entry, episode_ID_entry, timestamp_entry):
        # Get the values from text boxes
        scene_name = scene_name_entry.get()
        scene_desc = scene_desc_entry.get()
        episode_ID = episode_ID_entry.get()
        timestamp = timestamp_entry.get()

        try:
            # Establish connection between Python and MySQL database via connector API
            connection = connector.connect(
                host="localhost",
                user="DisenchantmentAdmin",  # use your own
                password=password_entry.get(),  # use the entered password
                database="testdisenchantment",  # set the default schema
                raise_on_warnings=True
            )

            # Create a cursor object to execute SQL statements
            cursor = connection.cursor()

            # Check if the scene_reference_name already exists
            cursor.execute("SELECT COUNT(*) FROM scene WHERE SceneName=%s", (scene_name,))
            result = cursor.fetchone()

            if result[0] == 0:
                # Call the AddScene procedure with the provided values
                cursor.callproc("AddScene", (scene_name, episode_ID, scene_desc, timestamp))
                connection.commit()
                print("Scene added successfully.")
                status_label.config(text="Scene added successfully.", fg="green")
                scene_name_entry.delete(0, tk.END)
                episode_ID_entry.delete(0, tk.END)
                scene_desc_entry.delete(0, tk.END)
                timestamp_entry.delete(0, tk.END)
            else:
                print("Scene name already exists.")
                status_label.config(text="Scene name already exists.", fg="red")

            # Fetch data from the 'scene' table
            cursor.execute("SELECT SceneID, SceneName, SceneDescription, EpisodeID, Timestamp FROM scene")

            # Insert the data into the tree view
            tree.delete(*tree.get_children())

            for row in cursor:
                tree.insert("", tk.END, values=row)

        except connector.Error as err:
            print(f"Failed to add scene: {err}")
            status_label.config(text="Failed to add scene.", fg="red")

        finally:
            # Close the cursor and the connection
            cursor.close()
            connection.close()



#----------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------------
def open_admin_scenes_window():
    admin_scenes_window = tk.Toplevel(window)
    admin_scenes_window.title("Admin Scenes")

    # Create the tree view
    tree = ttk.Treeview(admin_scenes_window, columns=("SceneID", "SceneName", "SceneDescription", "EpisodeID", "Timestamp"))
    tree.heading("SceneID", text="Scene ID")
    tree.heading("SceneName", text="Scene Name")
    tree.heading("SceneDescription", text="Description")
    tree.heading("EpisodeID", text="Episode ID")
    tree.heading("Timestamp", text="Timestamp")
    tree.pack()

    # Fetch data from the 'scene' table
    try:
        # Establish connection between Python and MySQL database via connector API
        connection = connector.connect(
            host="localhost",
            user="DisenchantmentAdmin",  # use your own
            password=password_entry.get(),  # use the entered password
            database="testdisenchantment",  # set the default schema
            raise_on_warnings=True
        )

        # Create a cursor object to execute SQL statements
        cursor = connection.cursor()

        # Fetch data from the 'scene' table
        cursor.execute("SELECT SceneID, SceneName, SceneDescription, EpisodeID, Timestamp FROM scene")

        # Insert the data into the tree view
        for row in cursor:
            tree.insert("", tk.END, values=row)

    except connector.Error as err:
        print(f"Error fetching data from the 'scene' table: {err}")

    finally:
        # Close the cursor and the connection
        cursor.close()
        connection.close()

    # Create frames for interactive text boxes and labels
    text_boxes_frame = tk.Frame(admin_scenes_window)
    text_boxes_frame.pack(pady=20)

    # Create the labels for text boxes
    scene_name_label = tk.Label(text_boxes_frame, text="Scene Name:")
    scene_name_label.grid(row=0, column=0)
    scene_desc_label = tk.Label(text_boxes_frame, text="Scene Description:")
    scene_desc_label.grid(row=1, column=0)
    episode_ID_label = tk.Label(text_boxes_frame, text="EpisodeID:")
    episode_ID_label.grid(row=2, column=0)
    timestamp_label = tk.Label(text_boxes_frame, text="Timestamp:")
    timestamp_label.grid(row=3, column=0)

    # Create the text boxes
    scene_name_entry = tk.Entry(text_boxes_frame)
    scene_name_entry.grid(row=0, column=1)
    scene_desc_entry = tk.Entry(text_boxes_frame)
    scene_desc_entry.grid(row=1, column=1)
    episode_ID_entry = tk.Entry(text_boxes_frame)
    episode_ID_entry.grid(row=2, column=1)
    timestamp_entry = tk.Entry(text_boxes_frame)
    timestamp_entry.grid(row=3, column=1)

    # Create the "Add" button
    add_button = tk.Button(admin_scenes_window, text="Add", command=lambda: add_scene(tree, scene_name_entry, scene_desc_entry, episode_ID_entry, timestamp_entry))
    add_button.pack()


    # Create the "Open Scene" button
    open_scene_button = tk.Button(admin_scenes_window, text="Open Scene", command=lambda: open_scene_elements(admin_scenes_window, tree, cursor, "testdisenchantment"))
    open_scene_button.pack()

    # Create the label for displaying status
    status_label = tk.Label(admin_scenes_window, text="")
    status_label.pack()

    def add_scene(tree, scene_name_entry, scene_desc_entry, episode_ID_entry, timestamp_entry):
        # Get the values from text boxes
        scene_name = scene_name_entry.get()
        scene_desc = scene_desc_entry.get()
        episode_ID = episode_ID_entry.get()
        timestamp = timestamp_entry.get()

        try:
            # Establish connection between Python and MySQL database via connector API
            connection = connector.connect(
                host="localhost",
                user="DisenchantmentAdmin",  # use your own
                password=password_entry.get(),  # use the entered password
                database="testdisenchantment",  # set the default schema
                raise_on_warnings=True
            )

            # Create a cursor object to execute SQL statements
            cursor = connection.cursor()

            # Check if the scene_reference_name already exists
            cursor.execute("SELECT COUNT(*) FROM scene WHERE SceneName=%s", (scene_name,))
            result = cursor.fetchone()

            if result[0] == 0:
                # Call the AddScene procedure with the provided values
                cursor.callproc("AddScene", (scene_name, episode_ID, scene_desc, timestamp))
                connection.commit()
                print("Scene added successfully.")
                status_label.config(text="Scene added successfully.", fg="green")
                scene_name_entry.delete(0, tk.END)
                episode_ID_entry.delete(0, tk.END)
                scene_desc_entry.delete(0, tk.END)
                timestamp_entry.delete(0, tk.END)
            else:
                print("Scene name already exists.")
                status_label.config(text="Scene name already exists.", fg="red")

            # Fetch data from the 'scene' table
            cursor.execute("SELECT SceneID, SceneName, SceneDescription, EpisodeID, Timestamp FROM scene")

            # Insert the data into the tree view
            tree.delete(*tree.get_children())

            for row in cursor:
                tree.insert("", tk.END, values=row)

        except connector.Error as err:
            print(f"Failed to add scene: {err}")
            status_label.config(text="Failed to add scene.", fg="red")

        finally:
            # Close the cursor and the connection
            cursor.close()
            connection.close()
#-----------------------------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------------------------
def open_trait_popup():
    trait_popup_window = tk.Toplevel()
    trait_popup_window.title("Add sceneaddresseselementhastrait")
def ADD_EPISODE(TREE, PART_ID_ENTRY, EPISODE_NAME_ENTRY, SYNOPSIS_ENTRY):
    # Get the values from the text boxes
    part_id = PART_ID_ENTRY.get()
    episode_name = EPISODE_NAME_ENTRY.get()
    synopsis = SYNOPSIS_ENTRY.get()

    # Establish connection between Python and MySQL database via connector API
    connection = connector.connect(
        host="localhost",
        user="DisenchantmentAdmin",  # Use your own
        password=PASSWORD_ENTRY.get(),  # Use the entered password
        database="testdisenchantment",  # Set the default schema
        raise_on_warnings=True
    )

    try:
        # Create a cursor object to execute SQL statements
        cursor = connection.cursor()

        # Call the stored procedure to add the episode
        ADD_EPISODE_PROCEDURE = "CALL AddEpisode(%s, %s, %s, %s)"
        cursor.execute(ADD_EPISODE_PROCEDURE, (part_id, episode_name, episode_name, synopsis))
        connection.commit()

        # Refresh the tree view to show the newly added episode
        REFRESH_TREE_VIEW(TREE)

    except connector.Error as err:
        print(f"Error adding episode: {err}")

    finally:
        # Close the cursor and the connection
        cursor.close()
        connection.close()

def REFRESH_TREE_VIEW(TREE):
    # Clear the existing items in the tree view
    TREE.delete(*TREE.get_children())

    # Fetch data from the 'episodes' table
    try:
        # Establish connection between Python and MySQL database via connector API
        connection = connector.connect(
            host="localhost",
            user="DisenchantmentAdmin",  # Use your own
            password=PASSWORD_ENTRY.get(),  # Use the entered password
            database="testdisenchantment",  # Set the default schema
            raise_on_warnings=True
        )

        # Create a cursor object to execute SQL statements
        cursor = connection.cursor()

        # Fetch data from the 'episodes' table
        cursor.execute("SELECT EpisodeID, EpisodeName, PartID, EpisodeName, Synopsis FROM episode")

        # Insert the data into the tree view
        for row in cursor:
            TREE.insert("", tk.END, values=row)

    except connector.Error as err:
        print(f"Error fetching data from the 'episodes' table: {err}")

    finally:
        # Close the cursor and the connection
        cursor.close()
        connection.close()
##-----------------------------------------------------------------------------------------------------------------------------
##-----------------------------------------------------------------------------------------------------------------------------

def open_scene_elements(admin_scenes_window, tree, cursor, database):
    # Establish connection between Python and MySQL database via connector API
    connection = connector.connect(
        host="localhost",
        user="DisenchantmentAdmin",  # use your own
        password=password_entry.get(),  # use the entered password
        database=database,  # use the specified database
        raise_on_warnings=True
    )

    try:
        # Get the selected item
        selected_item = tree.focus()
        # Get the SceneName associated with the selected item
        scene_name = tree.item(selected_item)['values'][1]

        # Create a new window
        scene_elements_window = tk.Toplevel(admin_scenes_window)
        scene_elements_window.title(f"Admin '{scene_name}' includes elements")

        # Create a frame for the buttons
        scene_elements_frame = tk.Frame(scene_elements_window)
        scene_elements_frame.pack()

        # Create the spreadsheet (treeview) in the new window
        spreadsheet = ttk.Treeview(scene_elements_frame)
        spreadsheet.pack()

        # Add columns to the spreadsheet
        spreadsheet['columns'] = ('ElementReferenceName', 'InclusionDescription', 'TableName')
        spreadsheet.heading('#0', text='ElementReferenceName')
        spreadsheet.heading('ElementReferenceName', text='Element Reference Name')
        spreadsheet.heading('InclusionDescription', text='Inclusion Description')
        spreadsheet.heading('TableName', text='Table Name')

        # Create a cursor object to execute SQL statements
        cursor = connection.cursor()

        # Get the SceneID associated with the selected item
        scene_id = tree.item(selected_item)['values'][0]

        # Query the sceneview view to get the elements for the selected SceneID
        query = f"SELECT ElementReferenceName, InclusionDescription, TableName FROM {database}.sceneview WHERE sceneview.SceneID = {scene_id}"
        cursor.execute(query)
        elements = cursor.fetchall()

        # Insert elements into the spreadsheet
        for element in elements:
            spreadsheet.insert('', 'end', values=element)

        # Fetch "ElementReferenceName" values from the "element" table
        cursor.execute("SELECT ElementReferenceName, ElementID FROM element")
        element_data = cursor.fetchall()
        element_names = [row[0] for row in element_data]
        element_ids = [row[1] for row in element_data]

        # Close the outer connection
        connection.close()

        # Create a label for the combobox
        element_label = tk.Label(scene_elements_frame, text="Element")
        element_label.pack()

        # Create a combobox for selecting element
        element_combobox = ttk.Combobox(scene_elements_frame, values=element_names, state="readonly")
        element_combobox.pack()

        # Create a label for the description entry
        description_label = tk.Label(scene_elements_frame, text="Description")
        description_label.pack()

        # Create an entry for entering description
        description_entry = tk.Entry(scene_elements_frame)
        description_entry.pack()

        # Create a label for the sequential ID entry
        sequential_id_label = tk.Label(scene_elements_frame, text="Sequential ID")
        sequential_id_label.pack()

        # Create an entry for entering sequential ID
        sequential_id_entry = tk.Entry(scene_elements_frame)
        sequential_id_entry.pack()

        def add_scene_includes_element():
            selected_element_index = element_combobox.current()
            if selected_element_index >= 0:
                new_element_id = element_ids[selected_element_index]
                new_description = description_entry.get()
                new_sequential_id = sequential_id_entry.get()

                # Establish a new connection within the function
                connection_inner = connector.connect(
                    host="localhost",
                    user="DisenchantmentAdmin",
                    password=password_entry.get(),
                    database=database,
                    raise_on_warnings=True
                )

                try:
                    # Create a new cursor for the inner connection
                    cursor_inner = connection_inner.cursor()

                    # Call the AddSceneIncludesElement procedure
                    add_scene_includes_element_procedure = f"""
                    CALL `{database}`.`AddSceneIncludesElement`({scene_id}, {new_element_id}, '{new_description}', {new_sequential_id});
                    """
                    cursor_inner.execute(add_scene_includes_element_procedure)
                    connection_inner.commit()

                    # Refresh the spreadsheet with updated data
                    spreadsheet.delete(*spreadsheet.get_children())
                    cursor_inner.execute(query)
                    updated_elements = cursor_inner.fetchall()
                    for element in updated_elements:
                        spreadsheet.insert('', 'end', values=element)

                    # Clear the entry fields
                    description_entry.delete(0, 'end')
                    sequential_id_entry.delete(0, 'end')

                finally:
                    # Close the inner connection
                    connection_inner.close()

        # Create a button to add scene includes element
        add_scene_includes_button = tk.Button(scene_elements_frame, text="Add scene includes element", command=add_scene_includes_element)
        add_scene_includes_button.pack()

    except connector.Error as error:
        print(f"Error connecting to the database: {error}")
        # Close the connection if an error occurs
        connection.close()


#------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------


# Create the main window
window = tk.Tk()
window.title("Main Window")

# Create a frame for the password label and entry
password_frame = tk.Frame(window)
password_frame.pack(pady=20)

# Create the password label
password_label = tk.Label(password_frame, text="Password:")
password_label.pack(side=tk.LEFT)

# Create the password entry widget
password_entry = tk.Entry(password_frame, show="*")
password_entry.pack(side=tk.LEFT)

# Create the "Connect" button
connect_button = tk.Button(window, text="Connect", command=connect_to_database)
connect_button.pack()

# Create the "Close Connection" button
close_button = tk.Button(window, text="Close Connection", command=close_connection)
close_button.pack()

# Create the connection status label
connection_label = tk.Label(window, text="Not Connected")
connection_label.pack()

# Start the Tkinter event loop
window.mainloop()


Connection between MySQL and Python is established.

